In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import io
import ipyleaflet
import json
from ipyleaflet import Map, Marker, LayersControl, SearchControl, FullScreenControl, WidgetControl, GeoJSON, LayerGroup, Popup, basemap_to_tiles, basemaps, TileLayer, Choropleth, ZoomControl
from branca.colormap import linear
from pandas_geojson import to_geojson
from ipywidgets import HTML, widgets, interactive, interact
from shapely.geometry import shape
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

In [20]:
year = 2015

# Downloading the csv file from the GitHub repository
url = "https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/coords.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
coords_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Printing out the first 5 rows of the dataframe
coords_df.head()
# Downloading the csv file from the GitHub repository
url = f"https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/{year}.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
data2015_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)
data2015 = load_data(
    f'https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/{year}.geo.json',
    f'{year}.geo.json',
     json.load)

In [21]:
import matplotlib.pyplot as plt

def plot_happiness_factors(dataframe, country_name):
    # Select the row for the specified country
    country_row = dataframe[dataframe['Country'] == country_name]

    if country_row.empty:
        print(f"No data found for {country_name}")
        return

    # Exclude non-numeric columns (e.g., Country and Region)
    numeric_columns = country_row.select_dtypes(include='number').drop(columns=['Happiness Rank', 'Happiness Score', 'Standard Error', 'Dystopia Residual'])

    # Plot all numeric columns on a single bar chart
    plt.figure(figsize=(12, 6))
    for column in numeric_columns.columns:
        plt.bar(column, country_row[column].values[0], label=column)
    
    plt.title(f"Happiness Factors for {country_name}")
    plt.xlabel("Factors")
    plt.ylabel("Values")
    plt.xticks(rotation=45, ha='right')
    plt.legend()
    plt.show()

# Get the unique country names from the DataFrame
country_names = data2015_df['Country'].unique()

# Create a dropdown widget
country_dropdown = widgets.Dropdown(
    options=country_names,
    value=country_names[0],
    description='Select Country:',
)

# Define the function to update the plot based on the selected country
def update_plot(selected_country):
    plot_happiness_factors(data2015_df, selected_country)

# Connect the dropdown widget to the update function
interact(update_plot, selected_country=country_dropdown)

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Assuming you have multiple DataFrames, you need to provide a list of them
dataframes = [data2015_df, data2016_df, data2017_df]  # Add more DataFrames as needed

# Get the unique country names from the selected DataFrame
def get_country_names(dataframe):
    return dataframe['Country'].unique()

# Function to plot happiness factors
def plot_happiness_factors(selected_dataframe, country_name):
    # Select the row for the specified country
    country_row = selected_dataframe[selected_dataframe['Country'] == country_name]

    if country_row.empty:
        print(f"No data found for {country_name}")
        return

    # Exclude non-numeric columns
    numeric_columns = country_row.select_dtypes(include='number').drop(columns=['Happiness Rank', 'Happiness Score', 'Standard Error', 'Dystopia Residual'])

    # Plot all numeric columns on a single bar chart
    plt.figure(figsize=(12, 6))
    for column in numeric_columns.columns:
        plt.bar(column, country_row[column].values[0], label=column)
    
    plt.title(f"Happiness Factors for {country_name}")
    plt.xlabel("Factors")
    plt.ylabel("Values")
    plt.xticks(rotation=45, ha='right')
    plt.legend()
    plt.show()

# Get the unique country names from the first DataFrame
default_dataframe = dataframes[0]
country_names = get_country_names(default_dataframe)

# Create dropdown widgets for selecting DataFrame and Country
dataframe_dropdown = widgets.Dropdown(
    options=dataframes,
    value=default_dataframe,
    description='Select DataFrame:',
)

country_dropdown = widgets.Dropdown(
    options=country_names,
    value=country_names[0],
    description='Select Country:',
)

# Define the function to update the plot based on the selected DataFrame and country
def update_plot(selected_dataframe, selected_country):
    plot_happiness_factors(selected_dataframe, selected_country)

# Connect the dropdown widgets to the update function
interact(update_plot, selected_dataframe=dataframe_dropdown, selected_country=country_dropdown)


In [23]:
dataframes_by_year = {
    2015: data2015_df,
    2016: data16_df,
    2017: data17_df,
    2018: data18_df,
    2019: data19_df,  
}

years = [2015, 2016, 2017, 2018, 2019]

In [24]:
# Create a dropdown widget for selecting the year
year_dropdown = widgets.Dropdown(
    options=years,
    value=years[0],
    description='Select Year:',
)

# Create a dropdown widget for selecting the country
country_dropdown = widgets.Dropdown(
    options=[],  # Options will be updated dynamically based on the selected year
    value=None,
    description='Select Country:',
)

# Function to update country dropdown based on the selected year
def update_countries(selected_year):
    # Assuming dataframes_by_year is a dictionary with years as keys and corresponding DataFrames as values
    countries = dataframes_by_year[selected_year]['Country'].unique()
    country_dropdown.options = countries
    country_dropdown.value = countries[0] if countries else None  # Set default value to the first country if available

# Connect the year dropdown to the update_countries function
interact(update_countries, selected_year=year_dropdown)

# Define the function to update the plot based on the selected country and year
def update_plot(selected_country, selected_year):
    # Get the DataFrame for the selected year
    selected_df = dataframes_by_year[selected_year]

    # Call the plot function with the updated DataFrame and selected country
    plot_happiness_factors(selected_df, selected_country, selected_year)

# Connect both dropdown widgets to the update_plot function
interact(update_plot, selected_country=country_dropdown, selected_year=year_dropdown)


interactive(children=(Dropdown(description='Select Year:', options=(2015, 2016, 2017, 2018, 2019), value=2015)…

interactive(children=(Dropdown(description='Select Country:', options=('Switzerland', 'Iceland', 'Denmark', 'N…

<function __main__.update_plot(selected_country, selected_year)>

In [ ]:
# Load the choropleth map
layer2015 = ipyleaflet.Choropleth(
    geo_data= data2015,
    choro_data=dict(zip(data2015_df['Country'].str[:2].str.upper().tolist(), data2015_df['Happiness Score'].tolist())),
    colormap=linear.YlOrRd_04,
    border_color='white',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'},
    name= '2015'
)

In [7]:
year = 2016

# Downloading the csv file from the GitHub repository
url = "https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/coords.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
coords_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Printing out the first 5 rows of the dataframe
coords_df.head()
# Downloading the csv file from the GitHub repository
url = f"https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/{year}cleaned.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
data16_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)
data16 = load_data(
    f'https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/{year}.geo.json',
    f'{year}.geo.json',
     json.load)

In [ ]:
# Load the choropleth map
layer2016 = ipyleaflet.Choropleth(
    geo_data= data16,
    choro_data=dict(zip(data16_df['Country'].str[:2].str.upper().tolist(), data16_df['Happiness_Score'].tolist())),
    colormap=linear.YlOrRd_04,
    border_color='white',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'},
    name= '2016'
)

In [8]:
year = 2017

# Downloading the csv file from the GitHub repository
url = "https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/coords.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
coords_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Printing out the first 5 rows of the dataframe
coords_df.head()
# Downloading the csv file from the GitHub repository
url = f"https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/{year}cleaned.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
data17_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)
data17 = load_data(
    f'https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/{year}.geo.json',
    f'{year}.geo.json',
     json.load)

In [ ]:
# Load the choropleth map
layer2017 = ipyleaflet.Choropleth(
    geo_data= data17,
    choro_data=dict(zip(data17_df['Country'].str[:2].str.upper().tolist(), data17_df['Happiness_Score'].tolist())),
    colormap=linear.YlOrRd_04,
    border_color='white',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'},
    name= '2017'
)

In [9]:
year = 2018

# Downloading the csv file from the GitHub repository
url = "https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/coords.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
coords_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Printing out the first 5 rows of the dataframe
coords_df.head()
# Downloading the csv file from the GitHub repository
url = f"https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/{year}cleaned.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
data18_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)
data18 = load_data(
    f'https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/{year}.geo.json',
    f'{year}.geo.json',
     json.load)

In [ ]:
# Load the choropleth map
layer2018 = ipyleaflet.Choropleth(
    geo_data= data18,
    choro_data=dict(zip(data18_df['Country'].str[:2].str.upper().tolist(), data18_df['Happiness_Score'].tolist())),
    colormap=linear.YlOrRd_04,
    border_color='white',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'},
    name= '2018'
)

In [10]:
year = 2019

# Downloading the csv file from the GitHub repository
url = "https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/coords.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
coords_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Printing out the first 5 rows of the dataframe
coords_df.head()
# Downloading the csv file from the GitHub repository
url = f"https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/{year}cleaned.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
data19_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)
data19 = load_data(
    f'https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/{year}.geo.json',
    f'{year}.geo.json',
     json.load)

In [ ]:
# Load the choropleth map
layer2019 = ipyleaflet.Choropleth(
    geo_data= data19,
    choro_data=dict(zip(data19_df['Country'].str[:2].str.upper().tolist(), data19_df['Happiness_Score'].tolist())),
    colormap=linear.YlOrRd_04,
    border_color='white',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'},
    name= '2019'
)

In [ ]:
# Search in a GeoJSON 

countries2015 = GeoJSON(data=data2015, style={'opacity': 0, 'fillOpacity': 0})

search2015 = SearchControl(
    position="topleft",
    layer=LayerGroup(layers=(countries2015,)),
    property_name="Country",
    zoom= 3
)

In [ ]:
# Search in a GeoJSON 

countries2016 = GeoJSON(data=data16, style={'opacity': 0, 'fillOpacity': 0})

search2016 = SearchControl(
    position="topleft",
    layer=LayerGroup(layers=(countries2016,)),
    property_name="Country",
    zoom= 3
)

In [ ]:
# Search in a GeoJSON 

countries2017 = GeoJSON(data=data17, style={'opacity': 0, 'fillOpacity': 0})

search2017 = SearchControl(
    position="topleft",
    layer=LayerGroup(layers=(countries2017,)),
    property_name="Country",
    zoom= 3
)

In [ ]:
# Search in a GeoJSON 

countries2018 = GeoJSON(data=data18, style={'opacity': 0, 'fillOpacity': 0})

search2018 = SearchControl(
    position="topleft",
    layer=LayerGroup(layers=(countries2018,)),
    property_name="Country",
    zoom= 3
)

In [ ]:
# Search in a GeoJSON 

countries2019 = GeoJSON(data=data19, style={'opacity': 0, 'fillOpacity': 0})

search2019 = SearchControl(
    position="topleft",
    layer=LayerGroup(layers=(countries2019,)),
    property_name="Country",
    zoom= 3
)

In [ ]:
# def add_popup2015(feature, **kwargs):
#     for layer in m.layers:
#         if isinstance(layer, Popup):
#             m.remove_layer(layer)
#     p = shape(feature["geometry"])
#     center = p.centroid.y, p.centroid.x
#     properties = feature["properties"]
#     message = HTML()
#     message.value = f"""<h4>{properties["Country"]}</h4>
#                         <b>Happiness Score: {properties["Happiness Score"]}
#                      """
#     popup2015 = Popup(
#         location=center,
#         child=message,
#         close_button=True,
#         auto_close=False,
#         close_on_escape_key=False,
#     )

#     m.add(popup2015)


# search2015.on_feature_found(add_popup2015)

In [ ]:
# # WORKING
# import ipywidgets as widgets
# from ipywidgets import interactive
# from ipyleaflet import Map, basemaps, TileLayer, basemap_to_tiles, Choropleth
# from IPython.display import display

# layer = ipyleaflet.Choropleth(
#                 geo_data= data,
#                 choro_data=dict(zip(data_df['Country'].str[:2].str.upper().tolist(), data_df['Happiness Score'].tolist())),
#                 colormap=linear.YlOrRd_04,
#                 border_color='white',
#                 style={'fillOpacity': 0.8, 'dashArray': '5, 5'},
#                 name= '2015'
#             )

# # Function to dynamically create and display the selected map with custom layer
# def display_map(selected_map):
#     map_to_display = ipyleaflet.Map(center = (0,0), zoom = 1)
#     map_to_display.clear()
#     if selected_map == 'CustomMap1':
#         # Create the Choropleth layer
#         map_to_display = ipyleaflet.Map(center = (0,0), zoom = 1)
#         map_to_display.add(layer)
#     elif selected_map == 'OpenTopoMap':
#         # Create the map with the OpenTopoMap basemap
#         map_to_display = Map(center=(0, 0), zoom=1, basemap=basemaps.OpenTopoMap)
#     else:
#         map_to_display = None

#     display(map_to_display)

# # Create an interactive widget with the display_map function and the dropdown
# interactive_map = interactive(display_map, selected_map={'CustomMap1': 'CustomMap1', 'OpenTopoMap': 'OpenTopoMap'})

# # Display the interactive widget
# display(interactive_map)

In [ ]:
def add_popup2015(feature, **kwargs):
    for layer in map_to_display.layers:
        if isinstance(layer, Popup):
            map_to_display.remove_layer(layer)
    p = shape(feature["geometry"])
    center = p.centroid.y, p.centroid.x
    properties = feature["properties"]
    message = HTML()
    message.value = f"""<h4>{properties["Country"]}</h4>
                        <b>Happiness Score: {properties["Happiness Score"]}
                     """
    popup2015 = Popup(
        location=center,
        child=message,
        close_button=True,
        auto_close=False,
        close_on_escape_key=False,
    )

    map_to_display.add(popup2015)

    plot_happiness_factors(data2015_df, {properties["Country"]})


search2015.on_feature_found(add_popup2015)

In [ ]:
def add_popup2016(feature, **kwargs):
    for layer in map_to_display.layers:
        if isinstance(layer, Popup):
            map_to_display.remove_layer(layer)
    p = shape(feature["geometry"])
    center = p.centroid.y, p.centroid.x
    properties = feature["properties"]
    message = HTML()
    message.value = f"""<h4>{properties["Country"]}</h4>
                        <b>Happiness Score: {properties["Happiness_Score"]}
                     """
    popup2016 = Popup(
        location=center,
        child=message,
        close_button=True,
        auto_close=False,
        close_on_escape_key=False,
    )

    map_to_display.add(popup2016)


search2016.on_feature_found(add_popup2016)

In [ ]:
def add_popup2017(feature, **kwargs):
    for layer in map_to_display.layers:
        if isinstance(layer, Popup):
            map_to_display.remove_layer(layer)
    p = shape(feature["geometry"])
    center = p.centroid.y, p.centroid.x
    properties = feature["properties"]
    message = HTML()
    message.value = f"""<h4>{properties["Country"]}</h4>
                        <b>Happiness Score: {properties["Happiness_Score"]}
                     """
    popup2017 = Popup(
        location=center,
        child=message,
        close_button=True,
        auto_close=False,
        close_on_escape_key=False,
    )

    map_to_display.add(popup2017)


search2017.on_feature_found(add_popup2017)

In [ ]:
def add_popup2018(feature, **kwargs):
    for layer in map_to_display.layers:
        if isinstance(layer, Popup):
            map_to_display.remove_layer(layer)
    p = shape(feature["geometry"])
    center = p.centroid.y, p.centroid.x
    properties = feature["properties"]
    message = HTML()
    message.value = f"""<h4>{properties["Country"]}</h4>
                        <b>Happiness Score: {properties["Happiness_Score"]}
                     """
    popup2018 = Popup(
        location=center,
        child=message,
        close_button=True,
        auto_close=False,
        close_on_escape_key=False,
    )

    map_to_display.add(popup2018)


search2018.on_feature_found(add_popup2018)

In [ ]:
def add_popup2019(feature, **kwargs):
    for layer in map_to_display.layers:
        if isinstance(layer, Popup):
            map_to_display.remove_layer(layer)
    p = shape(feature["geometry"])
    center = p.centroid.y, p.centroid.x
    properties = feature["properties"]
    message = HTML()
    message.value = f"""<h4>{properties["Country"]}</h4>
                        <b>Happiness Score: {properties["Happiness_Score"]}
                     """
    popup2019 = Popup(
        location=center,
        child=message,
        close_button=True,
        auto_close=False,
        close_on_escape_key=False,
    )

    map_to_display.add(popup2019)


search2019.on_feature_found(add_popup2018)

In [ ]:
# Call a map that's been created elsewhere WORKING

map_to_display = ipyleaflet.Map(center=(0,1), zoom = 1,)
mapnik = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)

# Function to dynamically create and display the selected map with custom layer
def display_map(selected_map):
    map_to_display.clear()
    map_to_display.add(mapnik)
    map_to_display.center=(0,0)
    map_to_display.zoom=1

    if selected_map == '2015':
        map_to_display.add(layer2015)
        map_to_display.add(search2015)
        
    elif selected_map == '2016':
        # Create the map with the OpenTopoMap basemap
        map_to_display.add(layer2016)
        map_to_display.add(search2016)
    elif selected_map == '2017':
        # Create the map with the OpenTopoMap basemap
        map_to_display.add(layer2017)
        map_to_display.add(search2017)
    elif selected_map == '2018':
        # Create the map with the OpenTopoMap basemap
        map_to_display.add(layer2018)
        map_to_display.add(search2018)
    elif selected_map == '2019':
        # Create the map with the OpenTopoMap basemap
        map_to_display.add(layer2019)
        map_to_display.add(search2019)
    else:
        map_to_display

    display(map_to_display)
# Create an interactive widget with the display_map function and the dropdown
interactive_map = interactive(display_map, selected_map=['2015', '2016', '2017', '2018', '2019'])

# Display the interactive widget
display(interactive_map)